<a href="https://colab.research.google.com/github/spindouken/atlas-machine_learning/blob/main/reinforcement_learning/deep_q_learning/Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project was finalized 1/10/2024.<br>
Author: Mason Counts

Resources:<br>
https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf<br>
https://gymnasium.farama.org/environments/atari/breakout/#breakout<br>
https://gymnasium.farama.org/api/wrappers/<br>
https://gymnasium.farama.org/api/env/<br>

# **DEEP Q-LEARNING WITH A DEEP Q-NETWORK (DQN)**

#### environment set-up

In [ ]:
import os
import tensorflow as tf
if tf.__version__!='2.11.0':
  !pip install tensorflow==2.11.0 --quiet
  os.kill(os.getpid(), 9)

In [ ]:
# gym[atari] now has ale-py integrated
!pip install gym[atari]

In [ ]:
pip install gym[accept-rom-license]

In [ ]:
pip install keras-rl2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 879.0 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### ***make gym env***

In [ ]:
# this is the format suggested by ALE dev blog post
import gym

env = gym.make('ALE/Breakout-v5',
    obs_type='rgb',                   # ram | rgb | grayscale
    frameskip=4,                      # frame skip
    mode=None,                        # game mode, see Machado et al. 2018
    difficulty=None,                  # game difficulty, see Machado et al. 2018
    repeat_action_probability=0.25,   # Sticky action probability
    full_action_space=False,          # Use all actions
    render_mode='rgb_array',          # None | human | rgb_array
)

# "The one keyword argument of note is render_mode.
#   To realize the benefits of natively rendering with SDL we must supply the render mode when constructing the environment.
#    We highly discourage using env.render()."

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


#### we do a little environment analysis.. is gud

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [ ]:
# Test the environment to see the observation
initial_observation = env.reset()

print(f"Initial Observation Type: {type(initial_observation)}")
print(f"Initial Observation Shape: {initial_observation.shape if hasattr(initial_observation, 'shape') else 'No shape attribute'}")


Initial Observation Type: <class 'numpy.ndarray'>
Initial Observation Shape: (210, 160, 3)


In [ ]:
print(f"Contents of the initial observation tuple: {initial_observation}")

Contents of the initial observation tuple: [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [ ]:
# Print action space details
print(f"Action Space: {env.action_space}")
print(f"Sample Action: {env.action_space.sample()}")

Action Space: Discrete(4)
Sample Action: 1


In [ ]:
# Print observation space details
print(f"Observation Space: {env.observation_space}")
print(f"Sample Observation: {env.observation_space.sample()}")

Observation Space: Box(0, 255, (210, 160, 3), uint8)
Sample Observation: [[[221  46 189]
  [130 197  36]
  [ 73  55 175]
  ...
  [ 88 205 169]
  [157 105 217]
  [ 44  70 204]]

 [[ 46 243 231]
  [252  61 199]
  [ 35 235  54]
  ...
  [ 91  88 251]
  [ 97 220 178]
  [ 11  69 142]]

 [[202  46 200]
  [ 70  93  78]
  [228   0  35]
  ...
  [116 188 183]
  [ 29 124 115]
  [112 239 121]]

 ...

 [[138 133 247]
  [ 97 187 180]
  [179 218  51]
  ...
  [186 247 156]
  [123  81  15]
  [ 72  56 112]]

 [[206 217 238]
  [138 171 153]
  [240  60  84]
  ...
  [ 24 191   6]
  [186 254 210]
  [ 57  87  87]]

 [[114 249 104]
  [236  32 130]
  [166  11  22]
  ...
  [ 57  97  75]
  [190 219   9]
  [  9 245  47]]]


In [ ]:
# Perform a sample action to see the next state, reward, done, info, and possibly truncated
action = env.action_space.sample()
result = env.step(action)

# Check the length of the result to handle both old and new step API formats
if len(result) == 4:
    next_state, reward, done, info = result
    truncated = None  # Old API does not return 'truncated'
else:
    next_state, reward, done, truncated, info = result

# Print details of the step
print(f"Next State Type: {type(next_state)}")
print(f"Next State Shape: {next_state.shape if hasattr(next_state, 'shape') else 'No shape attribute'}")
print(f"Reward: {reward}")
print(f"Done: {done}")
print(f"Truncated: {truncated}")
print(f"Info: {info}")

Next State Type: <class 'numpy.ndarray'>
Next State Shape: (210, 160, 3)
Reward: 0.0
Done: False
Truncated: None
Info: {'lives': 5, 'episode_frame_number': 4, 'frame_number': 4, 'rgb': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)}


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


#### **Imports and hyperparameters**

In [ ]:
import gym
import ale_py
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Activation, Permute
from tensorflow.keras.optimizers.legacy import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from PIL import Image
import numpy as np
from rl.core import Processor
import matplotlib as plt

In [ ]:
np.random.seed(123)
env.seed(123)

(693650678, 2973253328)

In [ ]:
# Hyperparameters
DQN_CONFIG = {
    'learning_rate': 1e-4,
    'training_steps': 60000,  # total number of steps to train for (including warmup)
    'input_shape': (84, 84),
    'memory_limit': 1000000,
    'window_length': 4,
    'nb_steps_warmup': 40000,  # number of steps to take before training begins
    'target_model_update': 20000,
    'enable_dueling_network': False,
    'dueling_type': None,
    # epsilon greedy action selection parameters below (policy)
    'eps_max': 1.0,
    'eps_min': 0.1,
    'eps_test': 0.05,
    'eps_nb_steps': 40000
}

#### **the code that does the preprocessing, indicates the policy, implements the DQN agent, and does the learnin and stuff**

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(DQN_CONFIG['input_shape']).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == DQN_CONFIG['input_shape']
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

    def process_info(self, info):
        """
        filter out non-scalar values from the info dictionary
        """
        scalar_info = {k: v for k, v in info.items() if np.isscalar(v)}
        return scalar_info

In [ ]:
nb_actions = env.action_space.n

# this is the same model that was described by Mnih et al. (2015) in
#   in Playing Atari with Deep Reinforcement Learning
#   http://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
input_shape = (DQN_CONFIG['window_length'],) + DQN_CONFIG['input_shape']

model = Sequential()
# (width, height, channels)
model.add(Permute((2, 3, 1), input_shape=input_shape))

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute_1 (Permute)         (None, 84, 84, 4)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 32)        8224      
                                                                 
 activation_5 (Activation)   (None, 20, 20, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_6 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_7 (Activation)   (None, 7, 7, 64)         

In [ ]:
# configure and compile the DQN agent with the model, sequential memory,
#  a linear annealed greedy eps greedy q policy with decay, and Adam learning rate.

memory = SequentialMemory(limit=DQN_CONFIG['memory_limit'], window_length=DQN_CONFIG['window_length'])

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=DQN_CONFIG['eps_max'],
                              value_min=DQN_CONFIG['eps_min'], value_test=DQN_CONFIG['eps_test'],
                              nb_steps=DQN_CONFIG['eps_nb_steps'])

dqn = DQNAgent(model=model, nb_actions=env.action_space.n, memory=memory, nb_steps_warmup=DQN_CONFIG['nb_steps_warmup'],
               enable_dueling_network=DQN_CONFIG['enable_dueling_network'], dueling_type=DQN_CONFIG['dueling_type'],
               target_model_update=DQN_CONFIG['target_model_update'], policy=policy,
               processor=AtariProcessor())

dqn.compile(Adam(learning_rate=DQN_CONFIG['learning_rate']), metrics=['mae'])


In [ ]:
# check new obervation space
observation = env.reset()
processed_observation = dqn.processor.process_observation(observation)
print(f"Processed Observation Shape: {processed_observation.shape}")

# run a single step to test
action = env.action_space.sample()
observation, reward, done, info = env.step(action)
print(f"Next Observation: {observation}, Reward: {reward}, Done: {done}, Info: {info}")


Processed Observation Shape: (84, 84)
Next Observation: [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]], Reward: 0.0, Done: False, Info: {'lives': 5, 'episode_frame_number': 4, 'frame_number': 4, 'rgb': array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


In [ ]:
weights_filename = '/content/drive/MyDrive/dqn/DQN_Breakout_Weights.h5'
checkpoint_weights_filename = '/content/drive/MyDrive/dqn_breakout_weights_{step}.h5f'
log_filename = '/content/drive/MyDrive/dqn_breakout_log.json'

# callbacks interval
interval = 50000

callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=interval)]
callbacks += [FileLogger(log_filename, interval=100)]

# train
training_steps = DQN_CONFIG['training_steps']
dqn.fit(env, callbacks=callbacks, nb_steps=training_steps, log_interval=10000, verbose=2)

# save the final weights after training completes
dqn.save_weights(weights_filename, overwrite=True)

Training for 60000 steps ...


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   173/60000: episode: 1, duration: 2.590s, episode steps: 173, steps per second:  67, episode reward:  1.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 1.520 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   451/60000: episode: 2, duration: 2.809s, episode steps: 278, steps per second:  99, episode reward:  3.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 1.518 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   678/60000: episode: 3, duration: 1.877s, episode steps: 227, steps per second: 121, episode reward:  2.000, mean reward:  0.009 [ 0.000,  1.000], mean action: 1.520 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   931/60000: episode: 4, duration: 2.147s, episode steps: 253, steps per second: 118, episode reward:  3.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 1.482 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1076/60000: episode: 5, duration: 1.139s, episode steps: 145, steps per second: 12

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 40107/60000: episode: 193, duration: 29.662s, episode steps: 275, steps per second:   9, episode reward:  1.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 0.593 [0.000, 3.000],  loss: 0.002629, mae: 0.043687, mean_q: 0.063893, mean_eps: 0.100000
 40304/60000: episode: 194, duration: 51.738s, episode steps: 197, steps per second:   4, episode reward:  0.000, mean reward:  0.000 [ 0.000,  0.000], mean action: 1.335 [0.000, 3.000],  loss: 0.002820, mae: 0.043621, mean_q: 0.061571, mean_eps: 0.100000
 40609/60000: episode: 195, duration: 77.474s, episode steps: 305, steps per second:   4, episode reward:  4.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 1.456 [0.000, 3.000],  loss: 0.003167, mae: 0.044038, mean_q: 0.059911, mean_eps: 0.100000
 40799/60000: episode: 196, duration: 48.936s, episode steps: 190, steps per second:   4, episode reward:  1.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 1.547 [0.000, 3.000],  loss: 0.003092, mae: 0.045522, mean_q: 0.062

#### **agent play while we evaluate it and record the gameplay (*we hopes*)**

In [ ]:
# not yet implemented
# purpose: for user to be able to pull the weights from this project's training
#  to evaluate the agent and/or continue training
!wget https://github.com/spindoken/atlasschool-machine_learning/reinforcement_learning/Deep_Q_Learning/policy.h5 -O policy.h5

code to evaluate agent without recording or rendering

In [ ]:
weights_filename = '/content/drive/MyDrive/dqn/DQN_Breakout_Weights.h5'

dqn.load_weights(weights_filename)

# evaluate for 10 episodes
dqn.test(env, nb_episodes=1, visualize=False)

ValueError: Layer count mismatch when loading weights from file. Model expected 5 layers, found 6 saved layers.

The code below will save the AI gameplay videos to the specified folder. A Video will be saved for each AI episode.<br>
The AI will utilize the weights filename that is specified in order to play the specified amount of episodes.
I was not able to get test to work properly with recording. I am unsure if this is due to a bug from using an odd combination of a certain tf version and other libraries or if I didn't properly initialize the agent for testing.

In [ ]:
import gym
from gym.wrappers import RecordEpisodeStatistics, RecordVideo

# wrap environment with the RecordEpisodeStatistics wrapper
env = RecordEpisodeStatistics(env)  # this will record the episode statistics

# wrap environment with RecordVideo wrapper
# this wrapper will record each episode of the agent playing
env = RecordVideo(env, '/content/drive/MyDrive/dqn/AIgameplayVODs', episode_trigger=lambda episode: True)

# load the trained weights
weights_file = '/content/drive/MyDrive/dqn/DQN_Breakout_Weights.h5'
dqn.load_weights(weights_file)

# evaluate AI
nb_episodes = 1  # number of runs for the AI to play
dqn.test(env, nb_episodes=1, visualize=False)

env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/dqn/AIgameplayVODs folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Testing for 1 episodes ...


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
Se

Episode 1: reward: 0.000, steps: 27000


code below will load a video from drive and play it in colab

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path):
    """
    workaround to display a video in colab

    Args:
        video_path: Path to the video file.
    """
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

video_path = '/content/drive/MyDrive/dqn/AIgameplayVODs/rl-video-episode-0.mp4'
play_video(video_path)



#### renderer dump
The code below is not currently functioning as intended.

In [ ]:
from rl.callbacks import Callback
import matplotlib.pyplot as plt
from IPython import display

class Render(Callback):
    def on_step_end(self, step, logs={}):
        plt.clf()
        plt.imshow(env.render(mode='rgb_array'))
        display.display(plt.gcf())
        display.clear_output(wait=True)

In [ ]:
dqn.test(env, nb_episodes=1, visualize=False, callbacks=[Render()])

NameError: name 'Render' is not defined

# **DEEP Q-LEARNING WITH A  DOUBLE DUELING Q-NETWORK (DDQN)**

## Full Environment Set-Up
This is includes installs, imports, config, lr scheduler, processing, policy, dqn, and model. <br>
Will have to run the first code block (tf version set-up) twice as it has a built in runtime reset built-in in order to reset tensorflow in environment.

In [ ]:
import os
import tensorflow as tf
# check if tf version is 2.11.0, if not: install it
if tf.__version__ != '2.11.0':
    !pip install tensorflow==2.11.0 --quiet
    os.kill(os.getpid(), 9)

# install all required libraries
!pip install gym[atari] gym[accept-rom-license] keras-rl2 --quiet

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 31.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation, Permute, Convolution2D, Input, Lambda
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from PIL import Image
import numpy as np
from rl.core import Processor
import gym
import ale_py
import matplotlib.pyplot as plt
from gym.wrappers import RecordEpisodeStatistics, RecordVideo

# make the gym enviornment using rgb mode
env = gym.make('ALE/Breakout-v5', obs_type='rgb', frameskip=4, mode=None, difficulty=None, repeat_action_probability=0.25, full_action_space=False, render_mode='rgb_array')

# random seeds for reproducibility
np.random.seed(123)
env.seed(123)

# hyperparameters for easy adjusting
DQN_CONFIG = {
    'initial_learning_rate': 1e-4,
    'decay_steps': 100000,
    'decay_rate': 0.96,
    'staircase': True,
    'input_shape': (84, 84),
    'memory_limit': 1000000,
    'window_length': 4,
    'target_model_update': 10000,
    'enable_dueling_network': True,
    'dueling_type': 'avg',
    # epsilon hyperparameters below
    'eps_max': 1.0,
    'eps_min': 0.1,
    'eps_test': 0.05,
    'eps_nb_steps': 250000,
    # training step hyperparameters below
    'training_steps': 500000,  # total training steps (including warmup)
    'nb_steps_warmup': 20000  # amount of steps to warmup for
}

# preprocessing observations
class AtariProcessor(Processor):
    """
    Processor for Atari games: processes the observation from the environment.
    """
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(DQN_CONFIG['input_shape']).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == DQN_CONFIG['input_shape']
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

    def process_info(self, info):
        """
        Filter out non-scalar values from the info dictionary
        """
        scalar_info = {k: v for k, v in info.items() if np.isscalar(v)}
        return scalar_info

# learning rate schedule setup
lr_schedule = ExponentialDecay(
    initial_learning_rate=DQN_CONFIG['initial_learning_rate'],
    decay_steps=DQN_CONFIG['decay_steps'],
    decay_rate=DQN_CONFIG['decay_rate'],
    staircase=DQN_CONFIG['staircase']
)

# implementing dueling DQN Model architecture
def build_dueling_dqn_model(input_shape, nb_actions):
    """
    Builds and returns a Dueling DQN model.
    """
    input_layer = Input(shape=(DQN_CONFIG['window_length'],) + input_shape)
    x = Permute((2, 3, 1))(input_layer)
    x = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(x)
    x = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(x)
    x = Convolution2D(64, (3, 3), strides=(1, 1), activation='relu')(x)
    x = Flatten()(x)

    # Dueling streams
    value_stream = Dense(512, activation='relu')(x)
    value = Dense(1, activation='linear')(value_stream)

    advantage_stream = Dense(512, activation='relu')(x)
    advantages = Dense(nb_actions, activation='linear')(advantage_stream)

    # combine streams
    def aggregate_streams(args):
        value, advantages = args
        return value + (advantages - tf.reduce_mean(advantages, axis=1, keepdims=True))

    output_layer = Lambda(aggregate_streams)([value, advantages])

    return Model(inputs=input_layer, outputs=output_layer)

# build dueling DQN model
model = build_dueling_dqn_model(DQN_CONFIG['input_shape'], env.action_space.n)
print(model.summary())

# DQN agent setup
memory = SequentialMemory(limit=DQN_CONFIG['memory_limit'], window_length=DQN_CONFIG['window_length'])
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=DQN_CONFIG['eps_max'], value_min=DQN_CONFIG['eps_min'], value_test=DQN_CONFIG['eps_test'], nb_steps=DQN_CONFIG['eps_nb_steps'])
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, memory=memory, nb_steps_warmup=DQN_CONFIG['nb_steps_warmup'], enable_dueling_network=DQN_CONFIG['enable_dueling_network'], dueling_type=DQN_CONFIG['dueling_type'], target_model_update=DQN_CONFIG['target_model_update'], policy=policy, processor=AtariProcessor())
dqn.compile(Adam(learning_rate=lr_schedule), metrics=['mae'])




/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 84, 84)]  0           []                               
                                                                                                  
 permute (Permute)              (None, 84, 84, 4)    0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 20, 20, 32)   8224        ['permute[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 9, 9, 64)     32832       ['conv2d[0][0]']                 
                                                                                              

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Initial Training

In [ ]:
# checkpoint indicator for saving weights during training
checkpoint_weights_filename = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_weights_{step}.h5'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]  # Save every 50,000 steps

# train
dqn.fit(env, nb_steps=DQN_CONFIG['training_steps'], callbacks=callbacks, verbose=2)

# save final weights
dqn.save_weights('/content/drive/MyDrive/dqn/BetaDDQN/dqn_final_weights.h5', overwrite=True)

## Evaluate Agent

In [ ]:
from gym.wrappers import RecordVideo

def evaluate_agent(env, dqn_agent, nb_episodes, video_folder):
    """
    test the DQN agent for a specified number of episodes and record videos
    """
    # wrap the environment to record videos
    env = RecordVideo(env, video_folder, episode_trigger=lambda episode: True)

    dqn_agent.test(env, nb_episodes=nb_episodes, visualize=False)

    env.close()

# load weights for the agent to utilize
weights_file = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_weights_150000.h5'
dqn.load_weights(weights_file)

# test the agent
evaluate_agent(env, dqn, nb_episodes=2, video_folder='/content/drive/MyDrive/dqn/BetaDDQN/dqn_evaluation_videos')

print("Evaluation completed.")


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/drive/MyDrive/dqn/BetaDDQN/dqn_evaluation_videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(


Testing for 2 episodes ...


/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 1: reward: 0.000, steps: 27000


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 2: reward: 0.000, steps: 27000
Evaluation completed.


In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path):
    """
    workaround to display a video in colab
    """
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

video_path = '/content/drive/MyDrive/dqn/BetaDDQN/training_videos (continued training of 150k step model)/rl-video-episode-74.mp4'
play_video(video_path)


In [ ]:
import os
from IPython.display import HTML, display
from base64 import b64encode

def play_videos_in_sequence(folder_path, speed=1.0):
    """
    play videos in sequence from a specified folder in google drive

    Args:
        folder_path: Path to the folder containing video files.
        speed: Playback speed (1.0 is normal speed, >1.0 is faster, <1.0 is slower).
    """
    # list all video files in the folder
    video_files = sorted([file for file in os.listdir(folder_path) if file.endswith('.mp4')])

    # display each video
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        mp4 = open(video_path, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        html = HTML("""
        <video width=400 controls autoplay loop>
            <source src="%s" type="video/mp4">
        </video>
        """ % data_url)
        display(html)

video_folder = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_evaluation_videos/'
play_videos_in_sequence(video_folder, speed=1.5)  # can adjust playback speed as needed


## Code for Training Continuation <br>
The purpose of the code below is to load the weights and memory files from the training in order to continue training.<br>
The code below was crafted to work with the full environmet set-up section. Meaning, some things like policy and and over config values are simly overwritten here before recompiling the dqn.

In [ ]:
!wget https://github.com/spindoken/atlasschool-machine_learning/reinforcement_learning/Deep_Q_Learning/policy.h5 -O policy.h5

In [ ]:
from gym.wrappers import RecordVideo, TimeLimit

# Function to wrap the environment
def wrap_env_for_recording(env, video_folder, max_video_length):
    env = TimeLimit(env, max_episode_steps=max_video_length)  # Limit the length of each episode
    env = RecordVideo(env, video_folder, episode_trigger=lambda episode: True)  # Record every episode
    return env

# Wrap the environment
video_folder = '/content/drive/MyDrive/dqn/BetaDDQN/300k_play_eps.2start'
max_video_length = 1500  # Set the maximum length of video recording (in steps)
env = wrap_env_for_recording(env, video_folder, max_video_length)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
import os
import pickle

# save and load replay memory (useful for retraining)
def save_replay_memory(dqn_agent, filename):
    with open(filename, 'wb') as f:
        pickle.dump(dqn_agent.memory, f)

def load_replay_memory(dqn_agent, filename):
    with open(filename, 'rb') as f:
        dqn_agent.memory = pickle.load(f)

nb_steps_warmup=5000 # didn't have replay memory saved before 200k, so have to warmup a bit (around 5% of total is good)
additional_training_steps = 100000  # total training steps to continue trainig for  (including warmup)

# Adjust the epsilon parameters for continued training (assuming linear decay in epsilon-greedy strategy)
epsilon_max_continued = 0.2  # New starting epsilon (adjust epsilon for what the value would have been at the end of the last training session.. or not, you do you)
epsilon_min_continued = 0.1
epsilon_test_continued = 0.05 # Test epsilon value

# Set up the policy with new epsilon values
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=epsilon_max_continued, value_min=epsilon_min_continued, value_test=epsilon_test_continued, nb_steps=additional_training_steps)

# Reinitialize the DQN agent with warmup steps
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, memory=memory, nb_steps_warmup=nb_steps_warmup,
               enable_dueling_network=DQN_CONFIG['enable_dueling_network'], dueling_type=DQN_CONFIG['dueling_type'],
               target_model_update=DQN_CONFIG['target_model_update'], policy=policy, processor=AtariProcessor())

# Recompile the agent
dqn.compile(Adam(learning_rate=lr_schedule), metrics=['mae'])

# Load the trained weights and memory
memory_file = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_memory(3rd_run).pkl'
weights_file = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_300k(3rd_run).h5'
dqn.load_weights(weights_file)

# Load weights and replay memory if they exist
if os.path.exists(weights_file):
    dqn.load_weights(weights_file)
if os.path.exists(memory_file):
    load_replay_memory(dqn, memory_file)

# Continue training
dqn.fit(env, nb_steps=additional_training_steps, verbose=2)

# save weights and replay memory after training
# remember to backup (and/or rename) this output weights file before continuing another training session or it will OVERWRITTEN!!!!
new_weights_file = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_new_weights.h5'
dqn.save_weights(new_weights_file, overwrite=True)
save_replay_memory(dqn, memory_file)


Training for 100000 steps ...


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=N

   728/100000: episode: 1, duration: 10.746s, episode steps: 728, steps per second:  68, episode reward: 14.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 1.889 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment

  1346/100000: episode: 2, duration: 6.800s, episode steps: 618, steps per second:  91, episode reward: 13.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.078 [0.000, 3.000],  loss: --, mae: --, mean_q: --, mean_eps: --


...

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment

 68348/100000: episode: 126, duration: 130.958s, episode steps: 585, steps per second:   4, episode reward: 12.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 1.771 [0.000, 3.000],  loss: 0.003432, mae: 0.633609, mean_q: 0.868071, mean_eps: 0.131945


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment

 69071/100000: episode: 127, duration: 162.618s, episode steps: 723, steps per second:   4, episode reward: 14.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 1.860 [0.000, 3.000],  loss: 0.003233, mae: 0.634627, mean_q: 0.869280, mean_eps: 0.131291


/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment ALE/Breakout-v5 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment

In [ ]:
new_weights_file = '/content/drive/MyDrive/dqn/BetaDDQN/dqn_new_weights.h5'
dqn.save_weights(new_weights_file, overwrite=True)

### Play Videos

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path):
    """
    workaround to display a video in colab

    Args:
        video_path: Path to the video file.
    """
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

video_path = '/content/drive/MyDrive/dqn/BetaDDQN/training_videos/rl-video-episode-0.mp4'
play_video(video_path)

In [ ]:
import os
from IPython.display import HTML, display
from base64 import b64encode

def play_videos_in_sequence(folder_path, speed=1.0):
    """
    play videos in sequence from a specified folder in google drive

    Args:
        folder_path: Path to the folder containing video files.
        speed: Playback speed (1.0 is normal speed, >1.0 is faster, <1.0 is slower).
    """
    # list all video files in the folder
    video_files = sorted([file for file in os.listdir(folder_path) if file.endswith('.mp4')])

    # display each video
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        mp4 = open(video_path, 'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        html = HTML("""
        <video width=400 controls autoplay loop>
            <source src="%s" type="video/mp4">
        </video>
        """ % data_url)
        display(html)

video_folder = '/content/drive/MyDrive/dqn/BetaDDQN/training_videos'
play_videos_in_sequence(video_folder, speed=1.5)  # Adjust speed as needed